In [213]:
import numpy as np
# import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt

In [214]:
class Layer:
    def sigmoid(x):
        return 1/(1+np.exp(-x))
        
    def sigmoid_derivative(x):
        return x * (1 - x)
    
    def relu(x):
        return np.maximum(0,x) 
    
    def relu_derivative(x):
        return 1. * (x > 0)

    def tanh(x):
        return np.tanh(x)

    def tanh_derivative(x):
        return 1 - x**2

    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def softmax_derivative(x):
        return Layer.softmax(x) * (1 - Layer.softmax(x))

    def cross_entropy(y_true, y_pred):
        return -np.sum(y_true * np.log(y_pred))

    def cross_entropy_derivative(y_true, y_pred):
        return -y_true/y_pred

    def mse(y_true, y_pred):
        return np.mean(np.square(y_true - y_pred))

    def mse_derivative(y_true, y_pred):
        return 2 * (y_pred - y_true) / y_true.size


In [215]:
class Dense:
    def __init__(self, input_dim, output_dim, activation):
        self.weights = np.random.randn(input_dim, output_dim) * np.sqrt(2.0 / (input_dim + output_dim))
        self.biases = np.zeros((1, output_dim))
        self.activation = activation
        self.activation_output = None
        self.error = None
        self.delta = None

    def forward_propagation(self, input):
        self.activation_output = self.activation(np.dot(input, self.weights) + self.biases)
        return self.activation_output

    def activation_derivative(self, x):
        if self.activation == Layer.sigmoid:
            return Layer.sigmoid_derivative(x)
        elif self.activation == Layer.relu:
            return Layer.relu_derivative(x)
        elif self.activation == Layer.tanh:
            return Layer.tanh_derivative(x)
        elif self.activation == Layer.softmax:
            return Layer.softmax_derivative(x)
    
    def get_weights(self):
        return self.weights
    
    def get_biases(self):
        return self.biases
   
    def __repr__(self):
        return "Dense Layer"

    def __str__(self):
        return "Dense Layer"
    
    def __call__(self, input):
        return self.forward_propagation(input)
        
    def __len__(self):
        return self.weights.shape[1]
    
    def __getitem__(self, index):
        return self.weights[:, index], self.biases[:, index]
    
    def __setitem__(self, index, value):
        self.weights[:, index] = value[0]
        self.biases[:, index] = value[1]
    
    def __delitem__(self, index):
        self.weights = np.delete(self.weights, index, axis=1)
        self.biases = np.delete(self.biases, index, axis=1)
    
    def __iter__(self):
        for i in range(self.weights.shape[1]):
            yield self.weights[:, i], self.biases[:, i]
    
    def __reversed__(self):
        for i in reversed(range(self.weights.shape[1])):
            yield self.weights[:, i], self.biases[:, i]
    
    def __contains__(self, item):
        return item in self.weights or item in self.biases
    
    def update(self, weights, biases):
        self.weights = weights
        self.biases = biases
    
    def save(self, path):
        np.savez(path, weights=self.weights, biases=self.biases)
    
    def load(self, path):
        data = np.load(path)
        self.weights = data['weights']
        self.biases = data['biases']
    
    def get_weights(self):
        return self.weights
     
    def get_biases(self):
        return self.biases
    
    def set_weights(self, weights):
        self.weights = weights
        self.biases = np.zeros((1, weights.shape[1]))

    def set_biases(self, biases):
        self.biases = biases
        self.weights = np.zeros((biases.shape[1], 1))

    def set_activation(self, activation):
        self.activation = activation
        


In [216]:
class Convo2D:
    def __init__(self, input_dim, output_dim, activation, filter_size, stride, padding):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activation = activation
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
        self.filters = np.random.randn(output_dim, input_dim, filter_size, filter_size) * np.sqrt(2.0 / (input_dim + output_dim))
        self.biases = np.zeros((1, output_dim))
        self.activation_output = None
        self.error = None
        self.delta = None

    def forward_propagation(self, input):
        self.input = input
        self.output_dim = int((self.input_dim - self.filter_size + 2 * self.padding) / self.stride + 1)
        self.activation_output = np.zeros((self.output_dim, self.output_dim, self.output_dim))
        for f in range(self.output_dim):
            for i in range(self.output_dim):
                for j in range(self.output_dim):
                    self.activation_output[f, i, j] = np.sum(self.input[:, i*self.stride:i*self.stride+self.filter_size, j*self.stride:j*self.stride+self.filter_size] * self.filters[f]) + self.biases[f]
        self.activation_output = self.activation(self.activation_output)
        return self.activation_output

    def activation_derivative(self, x):
        if self.activation == Layer.sigmoid:
            return Layer.sigmoid_derivative(x)
        elif self.activation == Layer.relu:
            return Layer.relu_derivative(x)
        elif self.activation == Layer.tanh:
            return Layer.tanh_derivative(x)
        elif self.activation == Layer.softmax:
            return Layer.softmax_derivative(x)
    
    def get_weights(self):
        return self.weights

    def get_biases(self):
        return self.biases
    
    def update(self, filters, biases):
        self.filters = filters
        self.biases = biases
    
    def save(self, path):
        np.savez(path, filters=self.filters, biases=self.biases)
    
    def load(self, path):
        data = np.load(path)
        self.filters = data['filters']
        self.biases = data['biases']

    def updateShape(self, input_dim):
        self.input_dim = input_dim
        self.output_dim = int((self.input_dim - self.filter_size + 2 * self.padding) / self.stride + 1)
        self.activation_output = np.zeros((self.output_dim, self.output_dim, self.output_dim))
    
        

In [217]:
class Sequential:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)
    
    def input(self, input):
        self.layers[0].input = input
        self.layers[0].output = input
        for i in range(1, len(self.layers)):
            self.layers[i].input = self.layers[i-1].output
            self.layers[i].output = self.layers[i].forward_propagation(self.layers[i].input)
        return self.layers[-1].output
    
    def output(self, output):
        self.layers[-1].output = output
        self.layers[-1].input = output
        for i in reversed(range(len(self.layers)-1)):
            self.layers[i].output = self.layers[i+1].input
            self.layers[i].input = self.layers[i].backward_propagation(self.layers[i].output)
        return self.layers[0].input

    def forward_propagation(self, input):
        activations = []
        input = input
        for layer in self.layers:
            activations.append(layer.forward_propagation(input))
            input = activations[-1]
        return activations

    def train(self, X, y, learning_rate, epochs):
        mses = []
        for i in range(epochs):
            sum_error = 0
            for j in range(len(X)):
                activation = self.forward_propagation(X[j])
                error = y[j] - activation[-1]
                self.backward_propagation(error, learning_rate)
                sum_error += np.sum(error)
            print('>epoch=%d, lrate=%.3f, error=%.3f' % (i, learning_rate, sum_error))
            mses.append(sum_error)
        return mses

    def predict(self, X):
        y_pred = []
        for i in range(len(X)):
            activation = self.forward_propagation(X[i])
            y_pred.append(activation[-1])
    
            red

In [218]:
def convolutional_neural_network():
    model = Sequential()
    model.add(Convo2D(1, 32, Layer.relu, 3, 1, 1))
    model.add(Convo2D(32, 64, Layer.relu, 3, 1, 1))
    model.add(Dense(64, 10, Layer.softmax))
    return model

def convo2d_test():
    model = convolutional_neural_network()
    X = np.random.randn(1, 28, 28)
    y = np.random.randn(1, 10)
    model.train(X, y, 0.1, 10)


In [219]:
class Pooling2D ():
    def __init__(self, input_dim, output_dim, activation, filter_size, stride, padding):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activation = activation
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
        self.activation_output = None
        self.error = None
        self.delta = None

    def forward_propagation(self, input):
        self.input = input
        self.output_dim = int((self.input_dim - self.filter_size + 2 * self.padding) / self.stride + 1)
        self.activation_output = np.zeros((self.output_dim, self.output_dim, self.output_dim))
        for f in range(self.output_dim):
            for i in range(self.output_dim):
                for j in range(self.output_dim):
                    self.activation_output[f, i, j] = np.max(self.input[:, i*self.stride:i*self.stride+self.filter_size, j*self.stride:j*self.stride+self.filter_size])
        self.activation_output = self.activation(self.activation_output)
        return self.activation_output


In [220]:
class Flatten ():
    def __init__(self, input_dim, output_dim, activation):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activation = activation
        self.activation_output = None
        self.error = None
        self.delta = None

    def forward_propagation(self, input):
        self.input = input
        self.activation_output = self.input.flatten()
        self.activation_output = self.activation(self.activation_output)
        return self.activation_output

    def save(self, path):
        np.savez(path, filters=self.filters, biases=self.biases)
    
    def load(self, path):
        data = np.load(path)
        self.filters = data['filters']
        self.biases = data['biases']

    def updateShape(self, input_dim):
        self.input_dim = input_dim
        self.output_dim = int((self.input_dim - self.filter_size + 2 * self.padding) / self.stride + 1)
        self.activation_output = np.zeros((self.output_dim, self.output_dim, self.output_dim))
        

In [221]:
import glob
from sklearn.model_selection import train_test_split
from sklearn.utils import as_float_array

np.random.seed(1000)
image_size = (100,100)

def read_image_array(path):
    path = glob.glob(path)
    images = []

    for image in path:
        img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size)
        images.append(img)
    return images

cats = read_image_array('./cats/*.jpg')
dogs = read_image_array('./dogs/*.jpg')

# Make an array of 0s for cats and 1s for dogs
X = np.array(cats + dogs)
y = np.array([0] * len(cats) + [1] * len(dogs))

# Normalize the data
X = X / 255.0 # 255 is the max value of a pixel
X = X.reshape(-1, 100, 100, 1) # -1 is for all images, 100x100 is the size of the image, 1 is for grayscale
y = y.reshape(-1, 1) # -1 is for all images, 1 is for grayscale

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
Sequential = Sequential() # Create the model
Sequential.add(Convo2D(1, 32, Layer.relu, 3, 1, 1)) # Add a convolutional layer with 32 filters, a 3x3 filter size and a relu activation function, with a stride of 1 and padding of 1 (so the output is the same size as the input)
Sequential.add(Convo2D(32, 64, Layer.relu, 3, 1, 1)) # Add another convolutional layer with 64 filters, a 3x3 filter size and a relu activation function, with a stride of 1 and padding of 1 (so the output is the same size as the input)
Sequential.add(Pooling2D(64, 64, Layer.relu, 2, 2, 0)) # Add a pooling layer with a 2x2 filter size and a stride of 2 
Sequential.add(Flatten(64, 10, Layer.softmax)) # Add a flatten layer to turn the 2D array into a 1D array so we can use it in a dense layer with a softmax activation function to get the probabilities for each class
Sequential.add(Dense(10, 1, Layer.sigmoid)) # Add a dense layer with 1 output (for the probability of the image being a dog) and a sigmoid activation function

Sequential.train(X_train, y_train, 0.1, 10) # Train the model with a learning rate of 0.1 and 10 epochs
pred = Sequential.predict(X_test)  # Predict the probabilities for each class
print(pred)  # Print the predictions

SyntaxError: invalid syntax (2806249540.py, line 43)